In [ ]:
using CSV, DataFrames, DataFrames, Statistics, LinearAlgebra, Random

In [ ]:
df_anycat = CSV.read("imatinib_data.csv", DataFrame)

In [ ]:
X_train = df_anycat[:, [:Max_tumor_size, :PRIM_MITOTIC_COUNT_NUMERIC, :SITE_CATEGORICAL_GASTRIC]]

train_treatments = df_anycat[:, :Ima_receipt]

train_recurred = convert(Vector{Bool}, df_anycat[:, :Primary_tumor_recurrence])

train_times = df_anycat[:, :Primary_tumor_RFS_years_ima_completion]*12;

In [ ]:
reward_lnr = IAI.CategoricalSurvivalRewardEstimator(
    propensity_estimator=IAI.RandomForestClassifier(show_progress=false, missingdatamode="separate_class"),
    outcome_estimator=IAI.RandomForestSurvivalLearner(show_progress=false, missingdatamode="separate_class"),
    reward_estimator=:direct_method,
    propensity_min_value=0.2,
    random_seed=123,
    evaluation_time= 84 #7 years
);

In [ ]:
train_predictions, train_reward_score = IAI.fit_predict!(reward_lnr,
    X_train, train_treatments, train_recurred, train_times,
    propensity_score_criterion = "misclassification",
    outcome_score_criterion = "harrell_c_statistic")
train_rewards = train_predictions[:reward];

In [ ]:
grid_rw = IAI.GridSearch(
    IAI.OptimalTreePolicyMaximizer(
        random_seed=123,
        max_categoric_levels_before_warning=20,
        missingdatamode="separate_class",
    ),
    max_depth=[3, 4, 5],
    minbucket=[5, 10, 15],
#     cp = [0.01, 0.05]
    cp = [0.001, 0.005]
)
IAI.fit!(grid_rw, X_train, train_rewards)